In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to C:\Users\girish
[nltk_data]     bhargav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\girish
[nltk_data]     bhargav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import tensorflow 
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents3.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

41 documents
11 classes ['Classes', 'Notice', 'clubs', 'departments', 'dispensary', 'goodbye', 'greeting', 'options', 'places_to_visit', 'sports', 'thanks']
93 unique lemmatized words ["'s", ',', 'a', 'all', 'anyone', 'are', 'attend', 'attraction', 'awesome', 'be', 'bhopal', 'bye', 'can', 'center', 'chatting', 'class', 'club', 'college', 'complex', 'could', 'day', 'depatments', 'dispensary', 'do', 'first', 'for', 'generally', 'get', 'give', 'go', 'good', 'goodbye', 'held', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hospital', 'how', 'i', 'if', 'in', 'is', 'join', 'later', 'list', 'lookup', 'manit', 'me', 'my', 'name', 'need', 'next', 'nice', 'notice', 'of', 'offered', 'on', 'one', 'patient', 'place', 'play', 'provide', 'regarding', 'see', 'should', 'sick', 'society', 'some', 'sport', 'student', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'tourist', 'treatment', 'update', 'visit', 'visited', 'what', 'where', 'will', 'year', 'you']


In [8]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
9/9 [==============================] - 1s 1ms/step - loss: 2.3771 - accuracy: 0.1220
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 2.3809 - accuracy: 0.0488
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 2.2088 - accuracy: 0.2439
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1378 - accuracy: 0.2927
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 1.9603 - accuracy: 0.3659
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 1.9359 - accuracy: 0.4390
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 1.8025 - accuracy: 0.5366
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 1.7528 - accuracy: 0.5366
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 1.5694 - accuracy: 0.5854
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 1.5793 - accuracy: 0.5854
Epoch 11/200
9/9 [===========

9/9 [==============================] - 0s 873us/step - loss: 0.1882 - accuracy: 0.9268
Epoch 84/200
9/9 [==============================] - 0s 997us/step - loss: 0.0810 - accuracy: 0.9756
Epoch 85/200
9/9 [==============================] - 0s 2ms/step - loss: 0.1926 - accuracy: 0.9268
Epoch 86/200
9/9 [==============================] - 0s 873us/step - loss: 0.1359 - accuracy: 0.9512
Epoch 87/200
9/9 [==============================] - 0s 1ms/step - loss: 0.1739 - accuracy: 0.9512
Epoch 88/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0702 - accuracy: 1.0000
Epoch 89/200
9/9 [==============================] - 0s 1ms/step - loss: 0.1350 - accuracy: 0.9268
Epoch 90/200
9/9 [==============================] - 0s 877us/step - loss: 0.0545 - accuracy: 1.0000
Epoch 91/200
9/9 [==============================] - 0s 993us/step - loss: 0.0503 - accuracy: 1.0000
Epoch 92/200
9/9 [==============================] - 0s 997us/step - loss: 0.0605 - accuracy: 0.9756
Epoch 93/200
9/9 [===

9/9 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 166/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0343 - accuracy: 0.9756
Epoch 167/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0177 - accuracy: 1.0000
Epoch 168/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0213 - accuracy: 1.0000
Epoch 169/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0168 - accuracy: 1.0000
Epoch 170/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0162 - accuracy: 1.0000
Epoch 171/200
9/9 [==============================] - 0s 997us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 172/200
9/9 [==============================] - 0s 997us/step - loss: 0.0166 - accuracy: 1.0000
Epoch 173/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0545 - accuracy: 1.0000
Epoch 174/200
9/9 [==============================] - 0s 1ms/step - loss: 0.1338 - accuracy: 0.9756
Epoch 175/200
9/9 [=

In [10]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents3.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [11]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [12]:
chatbot_response('Hello')

'Good to see you again'

In [13]:
chatbot_response('Where is dispensary in MANIT?')

'One should go to dispensary in MANIT located in front of sports complex for treatment purpose.'

In [14]:
chatbot_response('Where will my classes be held in first year?')

'To attend the classes in first year one should go to the New Teaching block.'

In [15]:
chatbot_response('What are some student clubs in MANIT?')

'Some student clubs in MANIT are IBC, VISION, ISTE, AAROHA, AE SE AENAK, ROBOTICS CLUB, QUIZZERS CLUB'

In [16]:
chatbot_response('Places to visit in Bhopal')

"Some places to visit in Bhopal are Upper Lake, Van Vihar National Park, Shahpura Lake, People's mall, Sair Sapata, Bharat Bhawan"

In [17]:
chatbot_response('List the sports I can play in MANIT')

'Sports that one can play in MANIT are Cricket, Football, Basketball, Volleyball, Badminton, Table Tennis, Chess'

In [18]:
chatbot_response('What are the departments in MANIT')

'The depaertments in MANIT are EE, CSE, ECE, MSME, CIVIL, CHEMICAL'

In [19]:
chatbot_response('Bye')

'See you!'

In [26]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "Query: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        speak(res)

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("MANIT Query Answering System")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="6", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Respond", width="10", height=2,
                    bd=0, bg="black", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-26-51a6a3fa0d2a>", line 18, in send
    speak(res)
NameError: name 'speak' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-26-51a6a3fa0d2a>", line 18, in send
    speak(res)
NameError: name 'speak' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-26-51a6a3fa0d2a>", line 18, in send
    speak(res)
NameError: name 'speak' is not defined


# Audio Query

In [27]:
import speech_recognition as s

def audioQuery():
    sr=s.Recognizer()
    sr.pause_threshold=1
    print("speak")
    with s.Microphone() as m:
        audio=sr.listen(m)
        query=sr.recognize_google(audio,language='eng-in')
        print(query)

In [28]:
audioQuery()

speak


UnknownValueError: 

# Speech Recognition

In [29]:
import pyttsx3 as pp
engine=pp.init()
voices=engine.getProperty('voices')
print(voices)

TypeError: item 1 in _argtypes_ passes a union by value, which is unsupported.

In [1]:
engine.setProperty('voice',voices[0].id)

def speak(word):
    engine.say(word)
    engine.runAndWait()

NameError: name 'engine' is not defined

In [ ]:
import speech_recognition in 